In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

# Load Trained Model

In [ ]:
import torch
model_path = './model_fold_0'
net = torch.load(model_path)
net.to("cpu")
net.eval()

In [ ]:
PATHS = []
PATHS.append(r"..\data\processed\kerryn_dec\M0.pickle")
PATHS.append(r"..\data\processed\kerryn_dec\M1.pickle")
PATHS.append(r"..\data\processed\kerryn_dec\M2.pickle")
         
NUM_WORKERS = 2
BATCH_SIZE = 6

In [ ]:
import torchvision.transforms as transforms
from utils import *
transforms = transforms.Compose([
    standardize_input()
    ])

# Load Data

In [ ]:
from torch.utils.data import DataLoader
import pickle
from MacDataset import MacDataset
raw_images = []
raw_labels = []
for i, path in enumerate(PATHS):
    path_data = pickle.load(open(path, "rb"))
    path_data["labels"][:] = i
    raw_images.append(path_data["images"])
    raw_labels.append(path_data["labels"])

images = np.vstack(raw_images)[:,:,:,:]
labels = np.hstack(raw_labels)

testset = MacDataset(images, labels, 
                            transform=transforms)
test_sampler = equal_classes_sampler(testset.labels)
test_loader = DataLoader(testset, batch_size=BATCH_SIZE, sampler=test_sampler,
                        shuffle=False, num_workers=0)

# Calculate CD80/CD206 Levels

In [ ]:
import torchvision
def imshow(img, transpose = True):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize = (11,4))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(test_loader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images[:,[0],:,:]))

l_names = ["M0", "M1", "M2"]
print('Ground Truth   : ', ' '.join('%15s' % l_names[int(labels[j].item())] 
                            for j in range(BATCH_SIZE)))

pred_names = ["M0", "M1", "M2"]
outputs = net(images[:,0:2,:,:].float())
predicted = torch.argmax(outputs,1).to(torch.double)
probability = torch.softmax(outputs, 1).detach().numpy()

print('Predicted      : ', ' '.join('%15s' % pred_names[int(predicted[j].item())] 
                            for j in range(BATCH_SIZE)))
print('M0 prob        : ', ' '.join('%15.5f' % probability[j][0].item()
                            for j in range(BATCH_SIZE)))

print('M1 prob        : ', ' '.join('%15.5f' % probability[j][1].item()
                            for j in range(BATCH_SIZE)))            
print('M2 prob        : ', ' '.join('%15.5f' % probability[j][2].item()
                            for j in range(BATCH_SIZE)))                                                          

In [ ]:
import seaborn as sn
import pandas as pd
def confusion_matrix(out, test_labels):
    # out = alveolar, marrow, monocyte
    # labels = alv
    num_classes_out = max(out) + 1
    num_classes_test = max(test_labels) + 1
    
    confusion_matrix = np.zeros((num_classes_out, num_classes_test))

    for t, p in zip(out, test_labels):
        confusion_matrix[int(t), int(p)] += 1

    matrix = np.array(confusion_matrix)
    matrix_2 = np.array([i/sum(i) for i in matrix])

    return matrix_2, matrix

def show_matrix(matrix, matrix_2, labels_index, labels_col):

    matrix_df = pd.DataFrame(matrix, index=labels_index, columns=labels_col)
    group_counts = ["{0:0.0f}".format(value) for value in
                matrix_2.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                     matrix.flatten()]
    class_labels = [f"{v1}\n{v2}" for v1, v2 in
          zip(group_percentages, group_counts)]

    class_labels = np.asarray(class_labels).reshape(matrix.shape)
    sn.set(font_scale=1.4) # for label size
    sn.heatmap(matrix_df, annot=class_labels, fmt="") # font size
    plt.title("Confusion Matrix of CNN on New Dataset")
    plt.show()
    

In [ ]:
correct = 0
total = 0
preds = np.array([])
truths = np.array([])
for data in test_loader:
    X, y = data[0][:,0:2,:,:].to("cpu"), data[1].to("cpu")
    pred = net(X.float())
    pred = torch.argmax(pred,1).detach().numpy()
    y = y.detach().numpy()
    preds = np.append(preds, pred)
    truths = np.append(truths, y)

In [ ]:
labels_index = ["M0", "M1", "M2"]
#labels_col = ["alveolar", "marrow", "alveolar_monocyte", "marrow_monocyte"]
labels_col = ["M0", "M1", "M2"]
matrix, numbers = confusion_matrix(preds.astype(int), truths.astype(int))
#show_matrix(matrix, labels_index, labels_col)
show_matrix(matrix, numbers, labels_index, labels_col)